In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
while not (PROJECT_ROOT / "src").exists() and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent


if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
import pandas as pd
import os
from IPython.display import HTML, display
from src.data_loader import main
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


In [3]:
main()

2025-10-20 03:17:54 - INFO - Fetched 4032 records from binance BTC/USDT (5m)
2025-10-20 03:17:54 - INFO - Saved bitcoin candlestick, MA, std dev, and signal data to C:/Users/rastr/Documents/trading-bot-project/data/temp\bitcoin_usd_14.csv
2025-10-20 03:17:56 - INFO - Completed bitcoin ingestion


In [4]:
train_df = pd.read_csv("C:/Users/rastr/Documents/trading-bot-project/data/train/train.csv")

In [5]:
display(train_df.tail(5))

,timestamp,open,high,low,close,sma_fast,sma_slow,ema_short,ema_long,rsi,...,sell,buy_score,sell_score,signal,position,entry_price,cash,qty,equity,simulation
4027,2025-10-20 02:55:00,108156.71,108167.52,108053.57,108053.58,108054.28,108026.44,108060.98,108386.82,45.05,...,False,1,0,NaN,1,108156.71,0.000000,0.658395,71141.900803,71281.082127
4028,2025-10-20 03:00:00,108053.57,108053.58,107900.56,107967.16,108057.00,107999.41,108046.55,108373.06,41.63,...,False,0,1,NaN,1,108156.71,0.000000,0.658395,71085.002336,71281.082127
4029,2025-10-20 03:05:00,107967.16,107978.60,107862.71,107903.20,108030.69,107981.25,108024.50,108357.65,39.25,...,False,0,1,NaN,1,108156.71,0.000000,0.658395,71042.891413,71281.082127
4030,2025-10-20 03:10:00,107903.19,108107.67,107854.71,108005.13,108032.74,107968.16,108021.52,108346.09,44.67,...,False,0,1,NaN,1,108156.71,0.000000,0.658395,71110.001581,71281.082127
4031,2025-10-20 03:15:00,108005.13,108466.65,107981.80,108373.90,108082.64,107980.44,108075.73,108347.00,58.95,...,True,1,2,SELL,0,NaN,71281.444985,0.000000,71281.444985,71281.444985


In [6]:
ts = pd.Timestamp('2025-10-15 19:00:00')
row = train_df.loc[pd.to_datetime(train_df['timestamp']) == ts]
display(row)

,timestamp,open,high,low,close,sma_fast,sma_slow,ema_short,ema_long,rsi,...,sell,buy_score,sell_score,signal,position,entry_price,cash,qty,equity,simulation
2780,2025-10-15 19:00:00,111186.77,111199.0,110826.21,110887.49,110785.48,110740.77,110798.15,111072.46,52.29,...,False,2,2,BUY,1,110887.49,0.0,0.703462,78005.135204,78083.218423
